In [ ]:
!nvidia-smi

In [ ]:
!python3 -m pip install --user Pillow

In [1]:
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

def conv_bn_relu(input_tensor, ch, kernel, padding="same", strides=1, weight_decay=5e-4):
    x = layers.Conv2D(ch, kernel, padding=padding, strides=strides,
                      kernel_regularizer=keras.regularizers.l2(weight_decay))(input_tensor)
    x = layers.BatchNormalization()(x)
    return layers.Activation("relu")(x)

def stem_block(input_tensor):
    x = conv_bn_relu(input_tensor, 32, 3, strides=2)
    branch1 = conv_bn_relu(x, 16, 1)
    branch1 = conv_bn_relu(branch1, 32, 3, strides=2)
    branch2 = layers.MaxPool2D(2)(x)
    x = layers.Concatenate()([branch1, branch2])
    return conv_bn_relu(x, 32, 1)

def dense_block(input_tensor, num_layers, growth_rate, bottleneck_width):
    x = input_tensor
    growth_rate = int(growth_rate / 2)

    for i in range(num_layers):
        inter_channel = int(growth_rate*bottleneck_width/4) * 4
        branch1 = conv_bn_relu(x, inter_channel, 1)
        branch1 = conv_bn_relu(branch1, growth_rate, 3)

        branch2 = conv_bn_relu(x, inter_channel, 1)
        branch2 = conv_bn_relu(branch2, growth_rate, 3)
        branch2 = conv_bn_relu(branch2, growth_rate, 3)
        x = layers.Concatenate()([x, branch1, branch2])
    return x

def transition_layer(input_tensor, k, use_pooling=True):
    x = conv_bn_relu(input_tensor, k, 1)
    if use_pooling:
        return layers.AveragePooling2D(2)(x)
    else:
        return x

def PeleeNet(input_shape=(224,224,3), use_stem_block=True, n_classes=1000):
    n_dense_layers = [3,4,8,6]
    bottleneck_width = [1,2,4,4]
    out_layers = [128,256,512,704]
    growth_rate = 32

    input = layers.Input(input_shape)
    x = stem_block(input) if use_stem_block else input
    for i in range(4):
        x = dense_block(x, n_dense_layers[i], growth_rate, bottleneck_width[i])
        use_pooling = i < 3
        x = transition_layer(x, out_layers[i], use_pooling=use_pooling)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(n_classes, activation="softmax")(x)
    return keras.models.Model(input, x)

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from PIL import Image
import pickle
import os
import shutil

EPOCHS = 120
BATCH_SIZE = 128
MODEL_VERSION = 1
MODEL_NAME = f'peleenet-cifar100-batch_size-{BATCH_SIZE}'
LOGS = 'logs'

model_directory = os.path.join(f'./{MODEL_NAME}')

if os.path.isdir(model_directory):
    pass
else:
    os.mkdir(model_directory)
    os.mkdir(os.path.join(model_directory, str(MODEL_VERSION)))
    os.mkdir(os.path.join(os.path.join(model_directory), str(MODEL_VERSION), LOGS))


def generator(X, y, batch_size, use_augmentation, shuffle, scale):
    if use_augmentation:
        base_gen = keras.preprocessing.image.ImageDataGenerator(
            horizontal_flip=True,
            width_shift_range=4.0/32.0,
            height_shift_range=4.0/32.0)
    else:
        base_gen = keras.preprocessing.image.ImageDataGenerator()
    for X_base, y_base in base_gen.flow(X, y, batch_size=batch_size, shuffle=shuffle):
        if scale != 1:
            X_batch = np.zeros((X_base.shape[0], X_base.shape[1]*scale,
                                X_base.shape[2]*scale, X_base.shape[3]), np.float32)
            for i in range(X_base.shape[0]):
                with Image.fromarray(X_base[i].astype(np.uint8)) as img:
                    img = img.resize((X_base.shape[1]*scale, X_base.shape[2]*scale), Image.LANCZOS)
                    X_batch[i] = np.asarray(img, np.float32) / 255.0
        else:
            X_batch = X_base / 255.0
        yield X_batch, y_base

# def lr_scheduler(epoch):
#     x = 0.4
#     if epoch >= 20: x /= 5.0
#     if epoch >= 50: x /= 5.0
#     if epoch >= 80: x /= 5.0
#     if epoch >= 100: x /= 5.0
#     return x

def train(use_augmentation, use_stem_block):
    #tf.compat.v1.logging.set_verbosity(tf.logging.FATAL)
    (X_train, y_train), (X_test, y_test) = keras.datasets.cifar100.load_data()
    y_train = keras.utils.to_categorical(y_train)
    y_test = keras.utils.to_categorical(y_test)
    
   
    # generator
    scale = 7 if use_stem_block else 1
    train_gen = generator(X_train, y_train, batch_size=BATCH_SIZE,
                          use_augmentation=use_augmentation, shuffle=True, scale=scale)
    test_gen = generator(X_test, y_test, batch_size=2000,
                         use_augmentation=False, shuffle=False, scale=scale)
        
    # network
    input_shape = (224,224,3) if use_stem_block else (32,32,3)
    model = PeleeNet(input_shape=input_shape, use_stem_block=use_stem_block, n_classes=100)
    model.compile(keras.optimizers.SGD(0.4, 0.9), "categorical_crossentropy", ["acc"])
    

    #scheduler = keras.callbacks.LearningRateScheduler(lr_scheduler)
    hist = keras.callbacks.History()
    tensorboard = keras.callbacks.TensorBoard(log_dir=f'./{MODEL_NAME}/{MODEL_VERSION}/{LOGS}', histogram_freq=20, write_graph=True, 
                                              write_images=False, update_freq='batch', profile_batch=2,
                                             embeddings_freq=20, embeddings_metadata=None)
    
    checkpoint = keras.callbacks.ModelCheckpoint(filepath=f'./{MODEL_NAME}/{MODEL_VERSION}/saved_model', monitor='val_loss', verbose=1, 
                                                 save_best_only=True, save_weights_only=False, mode='auto', 
                                                 save_freq='epoch')
    
    lr_plateau = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, 
                                                   verbose=1, mode='auto', cooldown=0, min_lr=0.0001)

    model.fit_generator(train_gen, steps_per_epoch=X_train.shape[0]//BATCH_SIZE,
                        validation_data=test_gen, validation_steps=X_test.shape[0]//1000,
                        callbacks=[lr_plateau, hist, tensorboard, checkpoint], epochs=EPOCHS, max_queue_size=1)
    
if __name__ == "__main__":
    train(True, True)

169009152/169001437 [==============================] - 7s 0us/step
Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 390 steps, validate for 10 steps
Epoch 1/120
389/390 [============================>.] - ETA: 0s - loss: 6.0712 - acc: 0.0687
Epoch 00001: val_loss improved from inf to 4.78452, saving model to ./peleenet-cifar100-batch_size-128/1/saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./peleenet-cifar100-batch_size-128/1/saved_model/assets
390/390 [==============================] - 190s 488ms/step - loss: 6.0669 - acc: 0.0687 - val_loss: 4.7845 - val_acc: 0.0459
Epoch 2/120
389/390 [============================>.] - ETA: 0s - loss: 3.8172 - acc: 0.1584
Epoch 00002: val_loss did not improve from 4.78452
390/390 [==============================] - 134s 344ms/step - loss: 3.8172 - acc: 0.1584 - val_loss: 8.4724 - val_ac